<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/ML/Recommender-System/Simple%20Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_rows', 1000)

In [2]:
import datetime
def dateparse (time_in_secs):
  try : 
    return datetime.datetime.fromtimestamp(float(time_in_secs))
  except:
    return datetime.datetime.fromtimestamp(float(0))

#Only for files uploaded from local drive

In [3]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
df=pd.read_csv('ratings.zip', sep=',', parse_dates=['timestamp'], date_parser=dateparse)
movies=pd.read_csv('movies.zip', sep=',')

# Only for small files from Git

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/sku1978/sk-share-repo/main/ML/Recommender-System/ratings.csv', sep=',', parse_dates=['timestamp'], date_parser=dateparse)
movies=pd.read_csv('https://raw.githubusercontent.com/sku1978/sk-share-repo/main/ML/Recommender-System/movies.csv', sep=',')

# Generic code

In [ ]:
df.head()

In [ ]:
movies.head()

In [ ]:
movies['genres_dict']=movies.apply(lambda row: row.genres.split('|'), axis=1).apply(lambda row: dict.fromkeys(row, 1))

In [ ]:
movie_classification=pd.concat([movies['title'],movies['genres_dict'].apply(pd.Series).drop('(no genres listed)', axis=1).fillna(0)], axis=1).set_index('title')

In [ ]:
movie_classification.head()

In [ ]:
df.info()

In [ ]:
movies.info()

In [ ]:
df=pd.merge(df,movies,on='movieId')

In [ ]:
df_movie=pd.DataFrame(df.groupby('title').rating.mean())
df_movie['num_of_rating']=pd.DataFrame(df.groupby('title').rating.count())

In [ ]:
df_movie.sort_values('num_of_rating', ascending=False).head()

In [ ]:
plt.style.use('dark_background')
df_movie.hist(column='rating', bins=70)

In [ ]:
df_movie.hist(column='num_of_rating', bins=100)

In [ ]:
df_movie[(df_movie['num_of_rating']>50)].hist(column='num_of_rating')

In [ ]:
sns.jointplot(data=df_movie, x='rating', y='num_of_rating')

In [ ]:
df_movie[(df_movie['num_of_rating']>50)]['num_of_rating'].head()

In [29]:
#limit movies to those watched by more than 50 users
df_filtered=df.merge(df_movie[(df_movie['num_of_rating']>50)]['num_of_rating'],on='title')

In [ ]:
moviemat=df_filtered.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
moviemat.head()

In [ ]:
#get a list of highest watched movies to test
df_movie.sort_values('num_of_rating', ascending=False).head(10)

,rating,num_of_rating
title,,
Forrest Gump (1994),4.164134,329
"Shawshank Redemption, The (1994)",4.429022,317
Pulp Fiction (1994),4.197068,307
"Silence of the Lambs, The (1991)",4.161290,279
"Matrix, The (1999)",4.192446,278
Star Wars: Episode IV - A New Hope (1977),4.231076,251
Jurassic Park (1993),3.750000,238
Braveheart (1995),4.031646,237
Terminator 2: Judgment Day (1991),3.970982,224


In [ ]:
movie_name="Pirates of the Caribbean: At World's End (2007)"
test_movie=moviemat[movie_name]

test_movie_class=movie_classification.dot(movie_classification.loc[movie_name]).rename('movie_class')
test_movie_corr=moviemat.corrwith(test_movie).rename('movie_corr_score')
test_movie_combined=pd.merge(test_movie_class, test_movie_corr, on='title')
test_movie_combined['combined_score']=test_movie_combined['movie_class']*test_movie_combined['movie_corr_score']

In [ ]:
test_movie_combined.sort_values('combined_score', ascending=False).head(10)

,movie_class,movie_corr_score,combined_score
title,,,
Pirates of the Caribbean: At World's End (2007),4.0,1.000000,4.000000
Pirates of the Caribbean: The Curse of the Black Pearl (2003),4.0,0.568120,2.272481
Pirates of the Caribbean: Dead Man's Chest (2006),3.0,0.714389,2.143167
"Mummy, The (1999)",4.0,0.456978,1.827913
Dragonheart (1996),3.0,0.419447,1.258341
Broken Arrow (1996),2.0,0.621516,1.243032
Bruce Almighty (2003),2.0,0.607207,1.214414
"Three Musketeers, The (1993)",3.0,0.396464,1.189392
Dogma (1999),3.0,0.383950,1.151851
